# Time Series: Assignment 4


##### Process Model
$$
\begin{aligned}
x_1 &= \bar x + w_1\\
x_t &= \phi x_{t-1} + w_t, \quad\text{for }t = 2,\ldots, T.
\end{aligned}
$$
where
$$w_1 = \epsilon_1,\quad w_t = \epsilon_t + \theta\epsilon_{t-1}, \quad t = 2,\ldots,T.$$ and $\epsilon_t$ are independent Gaussian variables with mean $0$ and variance $\sigma^2$.


##### Measurement Model
$$y_t = x_t + v_t,\quad \text{for }t = 1,\ldots,T,$$
where $v_t\sim N(0,0)$.



In [9]:
## Generate data

srand(123);
x̄ = randn();
σ = 0.1;
ϕ = 0.5;
θ = 0.2;

T = 100;
y = zeros(T);
ϵ = σ*randn(T);
y[1] = x̄ + ϵ[1];
for t = 2:T
    y[t] = ϕ*y[t-1] + θ*ϵ[t-1] + ϵ[t];
end

Our objective is to minimize $\frac{1}{2} ||\epsilon||^2$ where $\epsilon = (\epsilon_1,\epsilon_2,...,\epsilon_T$.  Note that $w_t = y_t - \phi y_{t-1}$ and $H w = \epsilon$ where 

$$H = 
\begin{bmatrix}
    1 & 0 & 0 & \dots  & 0 \\
    \theta & 1 & 0 & \dots  & 0 \\
     0 & \theta & 1 & \dots  & 0 \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
   0 & 0& 0 & \dots \theta & 1
\end{bmatrix}
$$.

With this observtion, let us form the objective function.


In [10]:
## Define objective function

function arma11_f(p)
    ϕ = p[1];
    θ = p[2];
    
    #---------------
    # Construct w(t)
    #---------------
    w = zeros(typeof(ϕ), T);
    w[1] = y[1] -  x̄ ;
    for t = 2:T
        w[t] = y[t] - ϕ*y[t-1]; 
    end
    
    #-----------------------------
    # Construct objective function.
    #-----------------------------
    H = [x == y+1 ? θ : (x == y ? 1 : 0) for x in 1:T, y in 1:T]
    ϵ = H\w
    return 0.5*sum(x->x^2, ϵ)
    
end

#----------------------------
# Automatic differentiation
#----------------------------

function arma11_g(p; h=1e-10)
    g = zeros(2);
    g[1] = imag(arma11_f([p[1] + im*h, p[2]]))/h;
    g[2] = imag(arma11_f([p[1], p[2] + im*h]))/h;
    return g
end

function arma11_h(p; h = 1e-10)
    H  = zeros(2,2);
    g1 = arma11_g(p);
    g2 = arma11_g([p[1] + h, p[2]]);
    g3 = arma11_g([p[1], p[2] + h]);
    H[1,1] = (g2[1] - g1[1])/h;
    H[2,2] = (g3[2] - g1[2])/h;
    H[1,2] = (g3[1] - g1[1])/h;
    H[2,1] = (g2[2] - g1[2])/h;
    return H
end


arma11_h (generic function with 1 method)

In [11]:
## Newton's method

srand(777);
itm = 100;
tol = 1e-6;
p   = rand(2) + 0.2;
g   = arma11_g(p);
h   = arma11_h(p);

#-----------------------------
# Implement Newton's method
#-----------------------------

for iter = 1:itm
    d = h\g;
    α = 1.0;
    p = p - α*d;
    g = arma11_g(p);
    h = arma11_h(p);
    
    obj = arma11_f(p);
    err = vecnorm(g);
    @printf("iter %3d, obj %1.5e, err %1.5e, ϕ %1.2f,  θ %1.5f\n", iter, obj, err, p[1], p[2]);
    err < tol && break;
end


iter   1, obj 6.24456e-01, err 7.48081e-02, ϕ 0.51,  θ 0.18720
iter   2, obj 6.22968e-01, err 2.49676e-03, ϕ 0.52,  θ 0.12802
iter   3, obj 6.22962e-01, err 1.93223e-05, ϕ 0.52,  θ 0.12328
iter   4, obj 6.22962e-01, err 1.00473e-10, ϕ 0.52,  θ 0.12326
